# Машинное обучение в потоке данных. Классификация

Реализация градиентного спуска, способного выполнять обработку и оценку качества в потоке данных. Версия для обработки большого массива данных на локальной машине.

In [5]:
import os

local_path = os.getcwd()
source = "covtype.data"

ram_shuffle(filename_in=local_path+"\\"+source, 
            filename_out=local_path+"\\shuffled_covtype.data", 
            header=False)

In [6]:
import csv, time
import numpy as np
from sklearn.linear_model import SGDClassifier

source = "shuffled_covtype.data"
SEP = ","
forest_type = [t+1 for t in range(7)]
SGD = SGDClassifier(loss="log", penalty=None, random_state=1, average=True)
accuracy = 0
holdout_count = 0
prog_accuracy = 0
prog_count = 0
cold_start = 200000
k_holdout = 10

with open(local_path+"\\"+source, "r") as R:   
    iterator = csv.reader(R, delimiter=SEP)
    
    for n, row in enumerate(iterator):
        if n > 250000: # Сокращение времени выполнения эксперимента
            break 
            
        # ОБРАБОТКА ДАННЫХ
        response = np.array([int(row[-1])]) # Откликом является последнее значение
        features = np.array(list(map(float,row[:-1]))).reshape(1,-1)  # см. описание ниже
        
        # МАШИННОЕ ОБУЧЕНИЕ
        if (n+1) >= cold_start and (n+1-cold_start) % k_holdout==0:
            if int(SGD.predict(features))==response[0]:
                accuracy += 1
            holdout_count += 1
            if (n+1-cold_start) % 25000 == 0 and (n+1) > cold_start:
                    print("%s точность на отложенных данных: %0.3f" 
                           % (time.strftime('%X'), accuracy / float(holdout_count)))
        else:
            
            # ПРОГРЕССИВНЫЙ КОНТРОЛЬ
            if (n+1) >= cold_start:
                if int(SGD.predict(features))==response[0]:
                    prog_accuracy += 1
                prog_count += 1
                if n % 25000 == 0 and n > cold_start:
                    print("%s точность с прогрессивным контролем: %0.3f" 
                           % (time.strftime("%X"), prog_accuracy / float(prog_count)))
                    
            # ФАЗА ОБУЧЕНИЯ
            SGD.partial_fit(features, response, classes=forest_type)
            
print("%s ИТОГОВАЯ точность на отложенных данных: %0.3f" 
       % (time.strftime('%X'), accuracy / ((n+1-cold_start) / float(k_holdout))))
print("%s ИТОГОВАЯ точность с прогрессивным контролем: %0.3f" 
       % (time.strftime("%X"), prog_accuracy / float(prog_count)))

19:33:10 точность на отложенных данных: 0.603
19:33:10 точность с прогрессивным контролем: 0.617
19:34:41 точность на отложенных данных: 0.610
19:34:41 точность с прогрессивным контролем: 0.617
19:34:41 ИТОГОВАЯ точность на отложенных данных: 0.610
19:34:41 ИТОГОВАЯ точность с прогрессивным контролем: 0.617
